In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8,4

### Import the input file received and analyze the format provided. 

Ideally recommend asking for data in the same format so that there is no reqork on codes

In [5]:
# getting the data
# Internal Data download

trend = pd.read_excel('C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\Internal Data\\Welder Monthly count Year 2015 to 2020.xlsx', sheet_name = 'Summary', skiprows = (0,1))
trend['Year'] = trend.Year.str.replace('FY','20')
trend = trend.apply(pd.to_numeric)
trend.head(6)

,Year,October,November,December,January,February,March,April,May,June,July,August,September
0,2015,23,21,19,13,11,11,8,9,8,9.0,11.0,12.0
1,2016,30,49,56,56,59,60,64,78,89,99.0,105.0,104.0
2,2017,105,105,106,107,94,82,74,67,69,73.0,74.0,80.0
3,2018,78,73,73,71,80,74,74,67,69,68.0,66.0,66.0
4,2019,64,76,95,103,109,132,145,113,102,103.0,108.0,112.0
5,2020,120,116,111,115,96,99,84,97,99,NaN,NaN,NaN


The historical data has been provided based on Fiscal year and not the calendar year, when we convert month and year into dates, we need to be careful to mark oct, nov and dec of a fiscal year (eg 2020) as the calendar year (2019).

In [7]:
#COnvert wide data to long
trend = pd.melt(trend, id_vars=['Year'], 
               value_vars=[ 'October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September'],
               var_name='Month', value_name='Labor Demand')
trend['Month'] = pd.to_datetime(trend.Month, format='%B').dt.month
trend.head(6)

,Year,Month,Labor Demand
0,2015,10,23.0
1,2016,10,30.0
2,2017,10,105.0
3,2018,10,78.0
4,2019,10,64.0
5,2020,10,120.0


Post coversion of fiscal year data into long format, we need to ensure that hte months of oct nov and dec( showing FY 15 for CY 14)  are  corrected

In [8]:
# the below function creates a new column with value -1 when the months are oct nov and dec. else the value is 0
def fn_name_def(fn_name):
 if fn_name['Month'] in (10, 11, 12):
   return -1
 return 0  

trend['Addition']=trend.apply(lambda fn_name: fn_name_def(fn_name),axis=1)  # applying the function

trend['New_Year'] = trend.Year + trend.Addition # making a new year column using the year addition indicator

trend.head(2)

,Year,Month,Labor Demand,Addition,New_Year
0,2015,10,23.0,-1,2014
1,2016,10,30.0,-1,2015


In [10]:
# converting date to date time format 
trend['DATE'] = pd.to_datetime((trend.New_Year*10000+trend.Month*100+1).apply(str),format='%Y%m%d')
trend = trend.dropna(how='any', axis='rows')
trend.dtypes
trend.head(6)

,Year,Month,Labor Demand,Addition,New_Year,DATE
0,2015,10,23.0,-1,2014,2014-10-01
1,2016,10,30.0,-1,2015,2015-10-01
2,2017,10,105.0,-1,2016,2016-10-01
3,2018,10,78.0,-1,2017,2017-10-01
4,2019,10,64.0,-1,2018,2018-10-01
5,2020,10,120.0,-1,2019,2019-10-01


In [11]:
# renaming columns and sorting index
trend['Month'] = pd.to_datetime(trend.DATE)
trend['data'] = trend['Labor Demand']
trend = trend.set_index('Month')
trend = trend.sort_index()
data = trend['data']
data.tail(20)

C:\Users\z0047cfr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\z0047cfr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Month
2018-11-01     76.0
2018-12-01     95.0
2019-01-01    103.0
2019-02-01    109.0
2019-03-01    132.0
2019-04-01    145.0
2019-05-01    113.0
2019-06-01    102.0
2019-07-01    103.0
2019-08-01    108.0
2019-09-01    112.0
2019-10-01    120.0
2019-11-01    116.0
2019-12-01    111.0
2020-01-01    115.0
2020-02-01     96.0
2020-03-01     99.0
2020-04-01     84.0
2020-05-01     97.0
2020-06-01     99.0
Name: data, dtype: float64

In [12]:
data.to_pickle('Labor_Demand.pkl')  # save a python data frame 
data.to_csv('Labor_Demand.csv') # Use Tab to seperate data